In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('dataset/marketing_campaign.csv', sep=';')
df.shape

In [ ]:
print(df.info())

In [ ]:
df.head()

In [ ]:
print(df.Z_CostContact.describe())
print(df.Z_Revenue.describe())

'Z_CostContact' and 'Z_Revenue' seem to be constants which won't affect our analysis or algorithms, so we can drop these columns.

In [ ]:
df = df.drop(['Z_Revenue', 'Z_CostContact'], axis=1)

In [ ]:
df['Kidhome'].value_counts()

In [ ]:
df['Teenhome'].value_counts()

The number of kids at home and teens at home can be combined into a single column.

In [ ]:
df['Minorhome'] = df.Kidhome + df.Teenhome
df = df.drop(['Kidhome', 'Teenhome'], axis=1)
# print(df.info())

In [ ]:
df['Minorhome'].value_counts()

In [ ]:
correlation = df.corr()
plt.figure(figsize=(14, 12))
sns.heatmap(correlation, center=0, vmax=0.5, square=True, linewidths=1, cmap='coolwarm')
# plt.show()
# df.corr()['Response'].sort_values(ascending=False)

In [ ]:
df.Dt_Customer.describe()

In [ ]:
df.Year_Birth.describe()

We can extract more useful features from these two columns and strip away the superfluous information.

In [ ]:
year = pd.to_numeric(df['Dt_Customer'].str.extract(r'(\d{4})', expand=False))
month = pd.to_numeric(df['Dt_Customer'].str.extract(r'-(\d{2})-', expand=False))

In [ ]:
month.value_counts()

In [ ]:
month_names = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}
month = month.value_counts().reset_index().sort_values('index')
month = month.replace(month_names)
print(month)

In [ ]:
month = month.rename(columns={'index': 'Month', 'Dt_Customer': 'Enrolments'})

In [ ]:
import plotly.express as px

In [ ]:
is_high = month['Enrolments'] > 200
color_sequence = is_high.apply(lambda x: 'crimson' if x else 'rgb(55, 83, 109)')
color_sequence = color_sequence.tolist()

In [ ]:
fig = px.bar(month, x='Month', y='Enrolments', title='Customer Enrolments by Month', color='Month', color_discrete_sequence=color_sequence)
fig.layout.update(showlegend=False)
fig.show()

This chart could have been much more powerful if we had data on the months in which the last few campaigns were active, so it would have been possible to measure impact. </br>Without this information and with our current visualization, there seem to be no stand-out months which could boost the response.

In [ ]:
df['Age'] = year - df['Year_Birth']
df = df.drop(['Year_Birth'], axis=1)
print(df['Age'].head())

In [ ]:
df.Age.describe()

In [ ]:
df.Age.sort_values(ascending=False)

In [ ]:
df = df[df['Age'] < 100]

Age makes more sense as a categorical variable so we can group users together.

In [ ]:
bins = [0, 25, 40, 50, 65, np.inf]
labels = ['<25', '25-40', '40-50', '50-65', '65+']
df['AgeRange'] = pd.cut(df['Age'], bins=bins, labels=labels)
df['AgeRange'].value_counts()

Let's identify the largest customer groups by age.

In [ ]:
df.columns

In [ ]:
rename_cols_dict = {'MntWines': 'Wines', 'MntFruits': 'Fruits', 'MntMeatProducts': 'Meat', 'MntFishProducts': 'Fish', 'MntSweetProducts': 'Sweet', 'MntGoldProds': 'Gold'}
df = df.rename(columns=rename_cols_dict)

In [ ]:
df['TotalSpend'] = df['Wines'] + df['Fruits'] + df['Meat'] + df['Fish'] + df['Sweet'] + df['Gold']

In [ ]:
grouped_df = df.groupby(by=['AgeRange'], as_index=False).agg({'TotalSpend': 'sum', 'Wines': 'sum', 'Fruits': 'sum', 'Meat': 'sum', 'Fish': 'sum', 'Sweet': 'sum', 'Gold': 'sum'})
grouped_df

In [ ]:
fig = px.bar(grouped_df, x='AgeRange', y='TotalSpend', title='Total Revenue by Age Group', color_discrete_sequence=['rgb(55, 83, 109)']*5)
fig.show()

Middle-aged customers (40+ years) form our largest buying group. This could be due to these customers being the primary breadwinners for the family.

In [ ]:
color_sequence = ['#521010', '#55ab18', '#b53128', '#9e34e0', '#2eb1d9', '#999912']
fig = px.bar(grouped_df, x='AgeRange', y=['Wines', 'Fruits', 'Meat', 'Fish', 'Sweet', 'Gold'], barmode='group', title='Product-wise Revenue by Age Group', color_discrete_sequence=color_sequence)
fig.layout.legend.update(title='Product')
fig.show()

Wines generate the most revenue across all customer age groups, followed by meat products. This indicates our fictional company might be a Wine & Meat company.

Handle Dt_Customer and Recency

In [ ]:
# print(df.loc[307:, ['Marital_Status', 'Income']].head(6))

In [ ]:
# from collections import Counter
# print(Counter(df['Education']))

In [ ]:
# %timeit -n 10000 df.Education.value_counts()
# %timeit -n 10000 Counter(df['Education'])

In [ ]:
edu_df = df.Education.value_counts().reset_index()
edu_df = edu_df.rename(columns={'index': 'Level', 'Education': 'Count'})
edu_df

In [ ]:
color_dict = {'PhD':'#791412', 'Master':'#a43741', 'Graduation':'#520810', 'Basic':'#a29994', '2n Cycle':'goldenrod'}
fig = px.pie(data_frame=edu_df, names='Level', values='Count', color='Level', color_discrete_map=color_dict, title='Distribution of Customers by Education Level')
fig.update_traces(pull=[0, 0, 0, 0.15, 0])
fig.show()

The Education column seems to be well-distributed with no extremely skewed groups.</br>
About half of our customers are graduates, with almost 98% having received more than basic education.</br>
An interesting observation is that 9% of our customers have received the "2n Cycle" education. This is a degree prevalent in European countries like Italy and Poland. So this segment possibly corresponds to our international customer base.

In [ ]:
df.Income.sort_values(ascending=False)

In [ ]:
df.Income.describe()

There seems to be a single high outlier for the income field. We can safely remove it due to the singularity. </br>We can also drop the (2240-2213)=27 rows with no entries since this number is less than 5% of the total number of data samples.

In [ ]:
df = df.dropna(subset=['Income'])
df = df[df['Income'] < 200000]

In [ ]:
fig = px.histogram(data_frame=df, x='Income',opacity=0.9, color_discrete_sequence=['#18acd9'], nbins=20, title='Distribution of Customers by Annual Income')
fig.show()

Most customers belong to the middle class, and have an annual household income between 30k and 70k. There are a few wealthy customers with annual incomes over 150k as well.

In [ ]:
df.Marital_Status.value_counts()

The Marital_Status column has some skewed groups. Most of these categories can be grouped together.

In [ ]:
mstatus_dict = {'Married':'Coupled', 'Together':'Coupled', 'Divorced':'Single', 'Widow':'Single', 'Alone':'Single', 'YOLO':'Single', 'Absurd':'Single'}
df['Marital_Status'] = df['Marital_Status'].replace(mstatus_dict)

In [ ]:
rel_df = df['Marital_Status'].value_counts().reset_index()
rel_df = rel_df.rename(columns={'index': 'Relationship Status', 'Marital_Status': 'Count'})

In [ ]:
color_map = {'Coupled': '#7a547a', 'Single': '#eb73af'}
fig = px.pie(data_frame=rel_df, names='Relationship Status', values='Count', title='Distribution of Customers by Relationship Status', color='Relationship Status', color_discrete_map=color_map)
fig.update_traces(pull=[0, 0.02])
fig.show()

In [ ]:
rename_cols_dict = {'Marital_Status': 'Rel_Status', 'NumWebPurchases': 'Web', 'NumCatalogPurchases': 'Catalog', 'NumStorePurchases': 'Store', 'NumDealsPurchases': 'Discount'}
df = df.rename(columns=rename_cols_dict)

In [152]:
df.columns

Index(['ID', 'Education', 'Rel_Status', 'Income', 'Dt_Customer', 'Recency',
       'Wines', 'Fruits', 'Meat', 'Fish', 'Sweet', 'Gold', 'Discount', 'Web',
       'Catalog', 'Store', 'NumWebVisitsMonth', 'AcceptedCmp3', 'AcceptedCmp4',
       'AcceptedCmp5', 'AcceptedCmp1', 'AcceptedCmp2', 'Complain', 'Response',
       'Minorhome', 'Age', 'AgeRange', 'TotalSpend'],
      dtype='object')

In [153]:
df.NumWebVisitsMonth.describe()

count    2212.000000
mean        5.321429
std         2.425597
min         0.000000
25%         3.000000
50%         6.000000
75%         7.000000
max        20.000000
Name: NumWebVisitsMonth, dtype: float64

In [154]:
df['TotalPurchases'] = df['Web'] + df['Catalog'] + df['Store'] + df['Discount']
df['SpendPerPurchase'] = df['TotalSpend'] / df['TotalPurchases']

In [178]:
fig = px.scatter(data_frame=df.query("Rel_Status=='Coupled'"), x='SpendPerPurchase', y='TotalSpend', color='Minorhome', size='Income', opacity=0.7)
fig.show()

In [181]:
fig = px.scatter(data_frame=df.query("Rel_Status=='Single'"), x='SpendPerPurchase', y='TotalSpend', color='Education', size='Minorhome', opacity=0.7)
fig.show()

In [ ]:
cluster_dataset = df[['Age', 'Education', 'Rel_Status', 'Income', 'TotalSpend', 'Wines', 'Fruits', 'Meat', 'Fish', 'Sweet', 'Gold']]

In [ ]:
cluster_dataset.head()

In [ ]:
cluster_dataset.to_csv('dataset/cluster_dataset.csv', index=False)

To be continued...